In [ ]:
#!pip install progress progressbar2 alive-progress tqdm

In [ ]:
import rioxarray
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import matplotlib.cm as cm
import h5py
import os
import pandas as pd
import collections, numpy
#from progressbar import progressbar
import time

In [ ]:
images = ['../data/Groningen_netcdf_20220109.nc',
          '../data/Groningen_netcdf_20220121.nc',
          '../data/Groningen_netcdf_20220202.nc',
          '../data/Groningen_netcdf_20220214.nc',
          '../data/Groningen_netcdf_20220226.nc',
          '../data/Groningen_netcdf_20220310.nc',
          '../data/Groningen_netcdf_20220322.nc']

In [ ]:
#a = xr.open_dataarray(images[0])

In [ ]:
#def plot_clipped(arr, per_min, per_max):
#    '''
# 
#    :param arr:
#    :param per_min:
#    :param per_max:
#    :return:
#    '''
#    return np.clip(arr, *np.percentile(arr, (per_min, per_max)))

In [ ]:
#plt.figure(figsize=(10,8))
#plt.subplot(121)
#plt.imshow(plot_clipped(a[...,0], 0.5, 99.5), cmap ='gray')
#plt.show()

In [ ]:
def get_data(x,c):
    # single file
    groningen_x1 = xr.open_dataarray(x)
    xarr = groningen_x1.to_numpy()
    
    # Save input as image
    #plt.imsave("../data/groningen_input"+str(c)+".tiff", xarr[:,:,0], cmap=cm.gray)
    
    xarr_df = np.reshape(xarr, (xarr.shape[0]*xarr.shape[1],xarr.shape[2]))
    xdf = pd.DataFrame(xarr_df, columns=groningen_x1.signatures.to_numpy())
    
    # Save input image as numpy array
    xx= xr.load_dataset(x)
    xx = xx.to_array()
    nump_x=np.array(xx).ravel()

    return groningen_x1,xdf

In [ ]:
def obtain_labels(xdf, th):
    a = 0
    b = 500000
    #b = len(xdf)
    print(xdf.head())
    xdf = xdf[a:b]
    print(xdf.head())
    xdf = xdf.reset_index()
    print(xdf.head())
    buildings = xdf['Buildings']
    
    railways = xdf['Railways']
    water = xdf['Water']
    roads = xdf['Roads']
    df_np=xdf.to_numpy()
    df_reference = []
    l = []
    for i in range(b-a):
        if i%1000==0:
            msg = "Loading " + str(i) + " images\n"
            with open('../log/log_file3.txt', 'a') as f:
                f.write(msg)
        if buildings[i] >= th:
            #a_label = [1,0,0,0,0]
            l.append(0)
            df_reference.append(df_np[i])
        if railways[i] >= th:
            #a_label = [0,1,0,0,0]
            l.append(1)
            df_reference.append(df_np[i])
        if water[i] >= th:
            #a_label = [0,0,1,0,0]
            l.append(2)
            df_reference.append(df_np[i])
        if roads[i] >= th:
            #a_label = [0,0,0,1,0]
            l.append(3)
            df_reference.append(df_np[i])
        if buildings[i] == 0 and railways[i] == 0 and water[i] == 0 and roads[i] ==0:
            #a_label = [0,0,0,0,1]
            l.append(4)
            df_reference.append(df_np[i])
        counter = collections.Counter(l)
    df_l_r = pd.DataFrame(df_reference,columns=list(xdf.columns))
    df_l_r['label']=l
    df_l_r.drop(['Buildings','Railways','Water','Roads'], axis=1)
    return l,counter, df_l_r

In [ ]:
def create_csv(xdf,c):
    #Save all features
    # Remove two columns name is 'C' and 'D'
    #xdf.drop(['Buildings','Railways','Water','Roads'], axis=1)
    #print(xdf.head())
    with open('../data3/reference_pixels_A_'+str(c)+'.csv', 'a') as f:
        xdf.to_csv(f, header=False)
    
    msg = "%%%%%%%%%%%%%%%Image " + str(c) + " done%%%%%%%%%%%%%%%"
    with open('../log/log_file3.txt', 'a') as f:
        f.write(msg)
    

In [ ]:
#xdf_=[None]*len(images)
#labels_=[None]*len(images)
#counter_=[None]*len(images)
#reshaped_=[None]*len(images)

xdf_=None
labels_=None
counter_=None
reshaped_=None
df_reference_=None
c=1
for x in images:
    print('#############################')
    groningen,xdf_ = get_data(x,c)
    s = time.time()
    labels_,counter_,df_reference_ = obtain_labels(xdf_,0.5)
    e = time.time()
    create_csv(df_reference_,c)
    c = c+1
    print(counter_)
    

In [ ]:
print(e-s)